In [73]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

### Set Up Mongo Connection

import time
import urllib.parse
from pymongo import MongoClient

username = urllib.parse.quote_plus('w210_db_user')
password = urllib.parse.quote_plus('q1w2e3r4$')
conn = MongoClient('mongodb://%s:%s@198.11.212.212:27017/w210_db' % (username, password))
db = conn.w210_db

pd.set_option('display.max_columns', 500)

In [74]:
result = db['sp_overlap_analysis'].find({'ind_cryptocompare': 1})

data = []
for i,row in enumerate(result):
    data.append(row)
    
master = pd.DataFrame(data)
print('Found %d records' % len(master))
master.head()

Found 453 records


,_id,ico_phase_ended,ind_coinmarketcap,ind_cryptocompare,ind_cryptowatch,ind_ravneeticoprice,name
0,5b40605c57c41002aa12d99a,1,1,1,1,0,PATRON
1,5b40605c57c41002aa12d9a3,1,0,1,0,0,Sentinel Protocol
2,5b40605c57c41002aa12d9cc,1,0,1,0,0,LocalCoinSwap
3,5b40605c57c41002aa12d9e1,1,0,1,0,0,VideoCoin
4,5b40605c57c41002aa12d9e4,1,0,1,0,0,Befund


In [75]:
result = db['sp_price_history'].find({})

price_history = []
for i,row in enumerate(result):
    price_history.append(row)
    
price_history[:2]

[{'_id': ObjectId('5b40625e57c41009c0b56bdc'),
  'name': 'PATRON',
  'symbol': 'PAT',
  'close_date': '2018-03-29',
  'price': 0},
 {'_id': ObjectId('5b40625e57c41009c0b56bdd'),
  'name': 'PATRON',
  'symbol': 'PAT',
  'close_date': '2018-03-30',
  'price': 0}]

In [77]:
api_url = 'https://min-api.cryptocompare.com/data/histoday?fsym={symbol}&tsym=USD&limit=10000'
for i,row in enumerate(master.to_dict(orient='records')):
    name = row['name']
    # Skip if we already have pricing data
#     if db['sp_price_history'].find_one({'name': name, 'source': 'CryptoCompare'}):
#         print('Found existing pricing data for {name}. Skipping...'.format(name=name))
#         continue
        
    try:
        # Look up symbol
        symbol = db['sp_cryptocompare'].find_one({'CoinName': name})['Symbol']
        # Get price data from CryptoCompare
        print('Getting Price Data from CryptoCompare for {name} - {symbol} ({i} of {length})'.format \
              (name=name,symbol=symbol,i=i+1,length=len(master)))
        response = requests.get(api_url.format(symbol=symbol))

        print('Writing records to db...')
        db.sp_price_history.delete_many({'symbol': symbol, 'source': 'CryptoCompare'})
        for r in response.json()['Data']:
            d = {
                'name': name,
                'symbol': symbol,
                'close_date' : datetime.strftime(datetime.fromtimestamp(r['time']),'%Y-%m-%d'),
                'price': float(r['close']),
                'source': 'CryptoCompare'
            }
            if d['price'] > 0:
                db.sp_price_history.insert_one(d)
    except:
        # Fail silently...
        pass
    time.sleep(1)

Getting Price Data from CryptoCompare for PATRON - PAT (1 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Sentinel Protocol - UPP (2 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for LocalCoinSwap - LCS (3 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for VideoCoin - VID (4 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Befund - BFDT (5 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Slate - SLX (6 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Chimaera - CHI (7 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Utrum - OOT (8 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Acorn Collective - OAK (9 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for VLUX - VLUX (10 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Tradelize - TDZ 

Getting Price Data from CryptoCompare for FundRequest - FND (109 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for CrowdCoinage - CCOS (110 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Doc Coin - DOC (111 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Sp8de - SPX* (112 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for LeadCoin - LDC (113 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for aXpire - AXP (117 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for KWHCoin - KWH (118 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Dropil - DROP (121 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Helbiz - HBZ (122 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Creditcoin - CRE* (123 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Elect

Writing records to db...
Getting Price Data from CryptoCompare for ILCoin - ILC (219 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Bounty0x - BNTY (220 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for MediBloc - MED* (222 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for SophiaTX - SPHTX (223 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Earth Token - EARTH (224 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for MilkCoin - MUU (225 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for BitDegree - BDG (226 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Refereum - RFR (227 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Modulum - MDL* (228 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Spectre - XSPEC (229 of 453)
Writing records to db...
Getting Price Da

Writing records to db...
Getting Price Data from CryptoCompare for KexCoin - KEX (329 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Authoreon - AUN (330 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Artemine - ARTE (331 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Matryx - MTX (332 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Cindicator - CND (333 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Anryze - RYZ (334 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Enigma - XNG (336 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Enigma - XNG (337 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for AirSwap - AST (338 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for DomRaider - DRT (339 of 453)
Writing records to db...
Getting Price Data from Crypto

Writing records to db...
Getting Price Data from CryptoCompare for Golos - GOLOS (447 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Incent - INCNT (448 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Komodo - KMD (449 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for Lykke - LKK (451 of 453)
Writing records to db...
Getting Price Data from CryptoCompare for deCLOUDs - DCS. (452 of 453)
Writing records to db...


In [66]:
result = db['sp_price_history'].find({})

data = []
for i,row in enumerate(result):
    data.append(row)
    
data

[{'_id': ObjectId('5b40625e57c41009c0b56bdc'),
  'name': 'PATRON',
  'symbol': 'PAT',
  'close_date': '2018-03-29',
  'price': 0},
 {'_id': ObjectId('5b40625e57c41009c0b56bdd'),
  'name': 'PATRON',
  'symbol': 'PAT',
  'close_date': '2018-03-30',
  'price': 0},
 {'_id': ObjectId('5b40625e57c41009c0b56bde'),
  'name': 'PATRON',
  'symbol': 'PAT',
  'close_date': '2018-03-31',
  'price': 0},
 {'_id': ObjectId('5b40625e57c41009c0b56bdf'),
  'name': 'PATRON',
  'symbol': 'PAT',
  'close_date': '2018-04-01',
  'price': 0},
 {'_id': ObjectId('5b40625e57c41009c0b56be0'),
  'name': 'PATRON',
  'symbol': 'PAT',
  'close_date': '2018-04-02',
  'price': 0},
 {'_id': ObjectId('5b40625e57c41009c0b56be1'),
  'name': 'PATRON',
  'symbol': 'PAT',
  'close_date': '2018-04-03',
  'price': 0},
 {'_id': ObjectId('5b40625e57c41009c0b56be2'),
  'name': 'PATRON',
  'symbol': 'PAT',
  'close_date': '2018-04-04',
  'price': 0},
 {'_id': ObjectId('5b40625e57c41009c0b56be3'),
  'name': 'PATRON',
  'symbol': 'PAT

In [63]:
datetime.strftime(datetime.fromtimestamp(1522281600),'%Y-%m-%d')

'2018-03-29'

In [29]:
import datetime
datetime.datetime.strftime(datetime.datetime.fromtimestamp(1522886400),'%Y-%m-%d')

'2018-04-05'